In [2]:
# Notebookをモジュールとして呼び出せる
import sys
import notebookutil as nbu
sys.meta_path.append(nbu.NotebookFinder())

import データ整形
import os
from bs4 import BeautifulSoup
from jannies_scraper import JanniesScraper
import pandas as pd
import glob
import re


<class 'numpy.int64'>
2019-05-17
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [3]:
# TODO
# モジュール呼び出しのために「データ整形」ファイルを「data_scraper」ファイルに移行
# 処理を関数化、引数は渡すべきパラメータとdataframe
# 帰り値は引数が追加されたdataframe

# フォルダ横断操作
# JanniesJrフォルダの全ての/日付/ticket.data[n].htmlを取得し前述の操作を行い
# CSV/ジャニーズJr/日付/ticket-data[n].csvとして書き出し 
# +日付ごとに結合して1枚のCSVに書き出し
# +全てを結合して一枚のCSVに書き出し

# さて、artiststrとdirstrを動的に生成したい
# 正規表現使えばよろしいかと
# と思ったがアーティスト名はトップ層にあるので大人しく配列で渡す

# アーティスト名のなかで各ディレクトリの名前を取得したい



# 先頭の./を取り除く ./2019-05-23 → 2019-05-23
def format_datename(name):
    return name.replace("./", "")

# ディレクトリの名前を変数に含めるときは、右端に/は含めないと規約づける。

def main():
#     事前準備：ルートを指定
    root_dir ="/Users/aki/ticket_tenbai" 
    os.chdir(f"{root_dir}/データ整形/") #クラス処理ごとにデータ整形ディレクトリに戻って欲しい
    artist_names = ["ジャニーズJr","セクシーゾーン", "キスマイフットツー", "NEWS"] #ジャニーズJrはもう取得したのでセクシーゾーンからやってもらう
    print("CSV生成を開始します")
    
    for artiststr in artist_names:
        print(f"{artiststr}のHTML取得→CSV変換を開始します")
        os.chdir(f"{root_dir}/{artiststr}")

#         アーティストディレクトリ内の全日付ディレクトリ名を取得
        date_foldername_list = sorted(glob.glob(f"{root_dir}/{artiststr}/*")) #['./2019-05-19', ... ]
        date_foldername_list = list(map(lambda x:format_datename(x),date_foldername_list )) #["2019-05-19",...]
        
#   '/Users/aki/ticket_tenbai/ジャニーズJr/2019-05-16'の中で
        for dirstr in date_foldername_list:
            print(f"{dirstr} フォルダ内のHTMLファイル→CSV変換を開始します")
            htmlpathes = sorted(glob.glob(f"{dirstr}/*")) #['./2019-05-19/ticket-data2.html'...]
            htmlpathes = list(filter(lambda x: "html" in x, htmlpathes))# /Users/aki/ticket_tenbai/ジャニーズJr/2019-05-16/ticket-data1.html こういうフルパスの集合            
            for pathstr in htmlpathes:                        
                if(os.path.isfile(pathstr)):
#                   dirstrからdatestrを正規表現で作る
                    datestr = extract_from_str(r"\d{4}-\d{2}-\d{2}",dirstr)
                    sc = JanniesScraper(datestr,pathstr)
                    sc.store_data()
#                   /Users/aki/ticket-tenbai/CSV/ジャニーズJr
                    save_dir_path = f"{root_dir}/CSV/{artiststr}/{datestr}"
                    #/Users/aki/ticket_tenbai/ジャニーズJr/2019-05-16/ticket-data1.html  →　ticket-data1
                    ticket_csv_name = extract_from_str(r"ticket-data\d{1,2}",pathstr)
                    os.makedirs(save_dir_path,exist_ok=True)
                    filename_saved = f"{save_dir_path}/{ticket_csv_name}.csv"
                    sc.write_in_csv(filename_saved)
                    if(os.path.isfile(filename_saved)):
                        print(f"CSVファイルをセーブしました：{filename_saved}")
                    else:
                        print("何か異常が起こっています")

                        
            print(f"{dirstr}:フォルダ内のHTMLファイル→CSV変換を完了しました")
                        
        print(f"{artiststr}のHTML取得→CSV変換を完了しました")
        
    print("全アーティストのHTML取得→CSV変換を完了しました")

        

        
                    
#                   print(f"{save_dir_path}/")
        



In [67]:
# /Users/aki/ticket_tenbai/ジャニーズJr/2019-05-16/ticket-data1.htmlから
# 1. 2019-05-16を抜き出す
# 2. ticket-data1を抜き出す
# →正規表現と対象文字列を与えて合致部分を抽出

def extract_from_str(reObj, targetstr):
    return re.search(reObj, targetstr).group()

# TODO
# Enka_scraperクラスを作成してインポート
# 　CSV結合クラスを作成
# 　日付ごと
# 　アーティストごと
# 　全件　の結合メソッドを生成

# ジャニーズ/演歌
# ジャニーズは/1アーティスト全日にち/全アーティスト1日にち全ファイル/全アーティスト全日にち/1アーティスト1日にち1ファイル
# 演歌は1ファイル/1日にち/全日にち

# 各クラスごとの処理として

from enka_scraper import EnkaScraper


class HTMLToCSVIterator:

    def __init__(self, artist_list): #, byday, byartist
        self.rootdir = "/Users/aki/ticket_tenbai" 
#         self.byday = byday
#         self.byartist = byartist
        self.artist_names =  artist_list#["ジャニーズJr","セクシーゾーン", "キスマイフットツー", "NEWS", "演歌"] 
        self.artist_name = artist_list[0]
        self.df = pd.DataFrame()
    
    
#     ユーティリティ

# 　　正規表現にマッチする部分文字列を抽出
    def extract_matched_str(reObj, targetstr):
        if re.search(reObj, targetstr):
            return re.search(reObj, targetstr).group()
        else:
            return None


    #クラス処理ごとにデータ整形ディレクトリに戻る
    def back_to_project_dir(self):
        os.chdir(f"{self.rootdir}/データ整形") 
        
        # 先頭の./を取り除く ./2019-05-23 → 2019-05-23
    def format_datename(name):
        return name.replace("./", "")

#   ロジック
        
#   TODO: アーティストリスト内の個別アーティストについて分析


#    アーティストディレクトリ内の全日付ディレクトリ名を取得
    def create_date_foldername_list(self):
        for artiststr in self.artist_names:
            self.back_to_project_dir()
            self.date_foldername_list = sorted(glob.glob(f"{self.rootdir}/{artiststr}/*")) #['self.rootdir/ジャニーズJr/2019-05-19', ... ]
            self.date_foldername_list = list(map(lambda x:x.replace("./", ""),self.date_foldername_list )) #["2019-05-19",...]
            self.date_foldername_list = list(filter(lambda x: (".csv" not in x),self.date_foldername_list ))
            self.fetch_all_HTML_in_date_dir(artiststr)

    #   '/Users/aki/ticket_tenbai/ジャニーズJr/2019-05-16'の中で
    def fetch_all_HTML_in_date_dir(self,artiststr):
        for dirstr in self.date_foldername_list:
            print(f"{dirstr}/フォルダ内のHTMLファイル→CSV変換を開始します")
            htmlpathes = sorted(glob.glob(f"{dirstr}/*")) #['./2019-05-19/ticket-data2.html'...]
#             フルパスの一つ
            for filepathstr in htmlpathes: 
                self.make_csv_from_HTML(filepathstr,dirstr,artiststr)
    
            
#          CSVファイル内のDataframe結合

# 　パスを与えるとインスタンスのDFと結合する
    def concat_dataframes(self, filename_saved):
        df_concated = pd.read_csv(filename_saved)
        if self.df.empty: #空っぽなら最初のだけコピー
            self.df = df_concated.copy()
        else:
#             from IPython.core.debugger import Pdb; Pdb().set_trace()
            self.df = pd.concat([self.df, df_concated],ignore_index=True)
#             from IPython.core.debugger import Pdb; Pdb().set_trace()
            print("結合しました：" +filename_saved)
        

#      htmlからcsv
    def make_csv_from_HTML(self,filepathstr,dirstr,artiststr):
#         ファイルなら
        if(os.path.isfile(filepathstr)):
            datestr = extract_from_str(r"\d{4}-\d{2}-\d{2}",dirstr) #2019-05-16 dirstrからdatestrを正規表現で作る
#     ジャニーズか演歌かで分岐
            if artiststr in ["ジャニーズJr","セクシーゾーン", "キスマイフットツー", "NEWS"]:
                sc = JanniesScraper(datestr,filepathstr)
            elif artiststr == "演歌":
                sc =    EnkaScraper(datestr,filepathstr)
            sc.store_data()   
            save_dir_path = f"{self.rootdir}/CSV/{artiststr}/{datestr}" #/Users/aki/ticket-tenbai/CSV/ジャニーズJr/2019-05-16
            ticket_csv_name = extract_from_str(r"ticket-data\d{1,2}",filepathstr) #/Users/aki/ticket_tenbai/ジャニーズJr/2019-05-16/ticket-data1.html  →　ticket-data1
            os.makedirs(save_dir_path,exist_ok=True)
            filename_saved = f"{save_dir_path}/{ticket_csv_name}.csv" #/Users/aki/ticket-tenbai/CSV/ジャニーズJr/2019-05-16/ticket-data1.csv
            sc.write_in_csv(filename_saved)
#           結合

#             from IPython.core.debugger import Pdb; Pdb().set_trace()
            self.concat_dataframes(filename_saved)
            
        if(os.path.isfile(filename_saved)):
            print(f"CSVファイルをセーブしました：{filename_saved} \n")
        else:
            print("何か異常が起こっています")

            
#     ファサード
    def main(self):
        self.create_date_foldername_list()
        
    
    
#     ジャニーズに関しては、対象のフォルダ内全部のファイルを結合するという動作だけ行いたい
# 連結→書き出しの動作
# 書き出しを抽出するべき to_csvとかでいいから
# 連結の範囲を決定するラッパーになるべき

# ちなみによく考えてみるとそんなもの必要なんだろうか
# ジャニーズ全体と演歌だけ結合すればよくないか？→と思ったけど、あったほうが良い

    def concat_per_day(self):
        dirlist = sorted(glob.glob(f"{self.rootdir}/CSV/{self.artist_name}/*/"))
        print(dirlist)
        for dirstr in dirlist:
            self.df = pd.DataFrame()
            htmlpathes = sorted(glob.glob(f"{dirstr}/*")) #['/Users/aki/ticket_tenbai/CSV/ジャニーズJr/2019-06-18/ticket-data8.csv''...]
            date = extract_from_str(r"\d{4}-\d{2}-\d{2}",dirstr)           
            htmlpathes = list(filter(lambda x: (f"{date}/{date}" not in x),htmlpathes ))

            for filepathstr in htmlpathes: 
                self.concat_dataframes(filepathstr)
            self.df.to_csv(f"{dirstr}/{date}.csv",index=0)
            print("日付DFを結合しました:" + f"{dirstr}{date}.csv")


    


#　アーティスト日付ごと→アーティスト全日付→全アーティストの順で行いたい
    def concat_per_artist(self):
        for artiststr in self.artist_names:
            self.back_to_project_dir()
            self.date_foldername_list = sorted(glob.glob(f"{self.rootdir}/CSV/{artiststr}/*")) #['/Users/aki/ticket_tenbai/CSV/ジャニーズJr/2019-07-21', ... ]
            self.date_foldername_list = list(filter(lambda x: (".csv" not in x),self.date_foldername_list ))
            self.df = pd.DataFrame()
            for dirstr in self.date_foldername_list:
                date = extract_from_str(r"\d{4}-\d{2}-\d{2}",dirstr) #2019-07-21
                self.concat_dataframes(f"{dirstr}/{date}.csv") #ジャニーズJr/2019-07-21/2019-07-21.csv
            self.df.to_csv(f"{self.rootdir}/CSV/{artiststr}/{artiststr}.csv",index=0)
            
            
        
            

In [68]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
root_dir ="/Users/aki/ticket_tenbai" 
arts  = glob.glob(f"{root_dir}/CSV/*")
arts = list(map(lambda x:x.replace(f"{root_dir}/CSV/", "") , arts))  #["ジャニーズJr", "NEWS" ,"セクシーゾーン", "キスマイフットツー", "演歌"]
dfs = []
for a in arts:
    hi = HTMLToCSVIterator([a])
    # hi.main()
    hi.concat_per_day()
    hi.concat_per_artist()
    
    print(f"{root_dir}/CSV/{a}/{a}.csv")
    d = pd.read_csv(f"{root_dir}/CSV/{a}/{a}.csv")
    d.shape
    dfs.append(d)
df = pd.concat(dfs,ignore_index=True) 
df.to_csv(f"{root_dir}/all-united.csv",index=0)

print("完了しました")

['/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-17/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-18/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-19/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-20/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-21/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-23/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-24/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-25/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-07/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-08/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-09/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-10/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-11/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-12/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-13/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-14/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-15/', '/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-16/', '/Users/aki

結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-21/ticket-data8.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-21/ticket-data9.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-21//2019-05-21.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data3.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data4.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data5.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data6.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data7.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data8.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/ticket-data9.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22//2019-05-22.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-23/ticket-data10.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-23/

日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-04//2019-07-04.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-05/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-05/ticket-data3.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-05//2019-07-05.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-06/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-06/ticket-data3.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-06//2019-07-06.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-07/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-07/ticket-data3.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-07//2019-07-07.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-08/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-08/ticket-data3.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-07-08//2019-07-08.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/

結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-08-11/ticket-data3.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-08-11/ticket-data4.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-08-11/ticket-data5.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-08-11/ticket-data6.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-08-11//2019-08-11.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-18/2019-05-18.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-19/2019-05-19.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-20/2019-05-20.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-21/2019-05-21.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-22/2019-05-22.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-23/2019-05-23.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-24/2019-05-24.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-05-25/2019-05-25.csv
結合しました：/Users/aki/ticket_tenbai/CSV/セクシーゾーン/2019-06-07/2019-06-07.csv
結合しました

日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-03//2019-06-03.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-04//2019-06-04.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-05//2019-06-05.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-06//2019-06-06.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-07//2019-06-07.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-08//2019-06-08.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-09//2019-06-09.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-10//2019-06-10.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-11//2019-06-11.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-12//2019-06-12.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-13//2019-06-13.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-14//2019-06-14.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-15//2019-06-15.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/演歌/2019-06-16//2019-06-16.csv
日付DFを結

...

結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data10.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data11.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data12.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data13.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data14.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data15.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data16.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data17.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data18.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data19.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data20.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-05/ticket-data21.csv
結合しました：/Users/aki/ticket_t

日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-10//2019-07-10.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data10.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data11.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data12.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data13.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data14.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data15.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data16.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data17.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data18.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data19.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-11/ticket-data20.csv
結合しました：/Users/aki/ticke

結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-19/ticket-data4.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-19/ticket-data5.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-19//2019-07-19.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-20/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-20/ticket-data3.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-20/ticket-data4.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-20/ticket-data5.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-20/ticket-data6.csv
日付DFを結合しました:/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-20//2019-07-20.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-21/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-21/ticket-data3.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-21/ticket-data4.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-21/ticket-data5.csv
結合しました：/Users/aki/ticket_tenba

結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data10.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data11.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data12.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data13.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data14.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data15.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data2.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data3.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data4.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data5.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data6.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data7.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/ticket-data8.csv
結合しました：/Users/aki/ticket_tenbai/

結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-22/2019-07-22.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-23/2019-07-23.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-24/2019-07-24.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-25/2019-07-25.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-28/2019-07-28.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-29/2019-07-29.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-30/2019-07-30.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-07-31/2019-07-31.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-01/2019-08-01.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-02/2019-08-02.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-03/2019-08-03.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-04/2019-08-04.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-05/2019-08-05.csv
結合しました：/Users/aki/ticket_tenbai/CSV/キスマイフットツー/2019-08-07/2019-08

In [78]:
d = pd.read_csv(f"{root_dir}/all-united.csv")


In [81]:
 d[d["is_jannies"]!= 1].shape

(1356, 11)

(625966, 11)